# Notebook to demonstrate the filtering process for the Pallas stream DTS data

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from utils import read_fmi_meteo_obs, plot_2D_dts_colormap, histogram_match, plot_dts_meteo_distributions

## Reading the DTS data (xr dataset) and air temperature observations (pd dataframe)

In [ ]:
file = '/Users/jpnousu/DTS_data/PROCESSED_DATA/pallas_dts_data.nc'
data = xr.open_dataset(file)

In [ ]:
meteo_file1 = r'/Users/jpnousu/DTS_data/AUX_DATA/Kittilä Kenttärova_ 1.7.2021 - 31.12.2023_a4ec5682-4462-40b6-990b-ff6ca553966e.csv'
meteo_file2 = r'/Users/jpnousu/DTS_data/AUX_DATA/Kittilä Lompolonvuoma_ 1.7.2021 - 31.12.2023_583c0ede-e628-46ef-a33b-b351ff859622.csv'

meteo1 = read_fmi_meteo_obs(filename=meteo_file1, resample='30MIN')
meteo2 = read_fmi_meteo_obs(filename=meteo_file2, resample='30MIN')

meteo1 = meteo1.rename(columns={'Tair': 'Kenttarova'})
meteo2 = meteo2.rename(columns={'Tair': 'Lompolo'})
meteo1['Lompolo'] = meteo2['Lompolo']

meteo1.plot(alpha=0.8)

## #1 Checking for clear erroneous timesteps in DTS data

In [ ]:
plt.figure(1)
data['T'].mean(dim='x').plot()
plt.figure(2)
data['T'].sel(time=slice('2023-11-01', None)).mean(dim='x').plot()

### Removing the dates after 2023-11-15

In [ ]:
data = data.sel(time=slice(None, "2023-11-15"))

## #2 Searching where the cable is out of water
### Are there extreme negative sections during winter? Yes.

In [ ]:
time_slice = slice('2021-12-01', '2022-03-01')
x_slice = slice(50, 2000)
plot_2D_dts_colormap(xr_data=data, meteo_df=meteo1, vmin=-10, time_slice=time_slice, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-02-01', '2022-02-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

### Are there sections where the summertime diurnal variation is close to observed air temperature? Yes.

In [ ]:
time_slice = slice('2022-07-01', '2022-08-01')
x_slice = slice(50, 2000)
plot_2D_dts_colormap(xr_data=data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-08-01', '2022-08-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

### Removing the data locations (x) where at least 5% of the data is negative

In [ ]:
# Copy of the data for filtering
f_data = data.copy()

# Compute the quantile along the time dimension
quantile = data["T"].quantile(0.05, dim="time")

# Identify locations where the quantile is below 0
mask = quantile < 0

# Replace all values at those locations with NaN
f_data["T"] = data["T"].where(~mask, np.nan)

#### Plotting the same figures to see if the filtering worked

In [ ]:
time_slice = slice('2021-12-01', '2022-03-01')
x_slice = slice(50, 2000)

plot_2D_dts_colormap(xr_data=f_data, meteo_df=meteo1, vmin=-10, time_slice=time_slice, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-02-01', '2022-02-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=f_data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

In [ ]:
time_slice = slice('2022-07-01', '2022-08-01')
x_slice = slice(50, 2000)
plot_2D_dts_colormap(xr_data=f_data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-08-01', '2022-08-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=f_data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

### Filtering worked mostly, but it seems that there are still DTS sections where summer diurnal variations are close to air temperature
#### For example here

In [ ]:
time_slice = slice('2022-06-08', '2022-06-10')
x_slice = slice(200, 1000)
plot_2D_dts_colormap(xr_data=f_data, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice)

#### Subsetting for summer months and common dates

In [ ]:
f_filtered = f_data.where((f_data.time.dt.month > 5) & (f_data.time.dt.month < 9), drop=True)

# Find common times
common_times = meteo1.index.intersection(f_filtered.time.values)

# Subset both datasets to only include common times
f_filtered = f_filtered.sel(time=common_times)
meteo1_filtered = meteo1.loc[common_times]

### Example distributions in summer
#### Close to air temperature distributions

In [ ]:
x_temp = 602.397

#plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Lompolo'], bins=50, alpha=0.5, label='Lompolo')
plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Kenttarova'], bins=50, alpha=0.5, label='Kenttarova')
plt.hist(f_filtered.sel(x=x_temp)['T'], bins=50, alpha=0.5, label='DTS')
hm_temp = histogram_match(np.array(f_filtered.sel(x=x_temp)['T']), np.array(meteo1_filtered['Kenttarova']), lims=[0,30])
plt.title(f'X = {x_temp}, histogram match = {hm_temp}')
plt.legend()

#### More realistic compared to air temperature distributions

In [ ]:
x_temp = 1612.933

#plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Lompolo'], bins=50, alpha=0.5, label='Lompolo')
plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Kenttarova'], bins=50, alpha=0.5, label='Kenttarova')
plt.hist(f_filtered.sel(x=x_temp)['T'], bins=50, alpha=0.5, label='DTS')
hm_temp = histogram_match(np.array(f_filtered.sel(x=x_temp)['T']), np.array(meteo1_filtered['Kenttarova']), lims=[0,30])
plt.title(f'X = {x_temp}, histogram match = {hm_temp}')
plt.legend()

#### Calculating and plotting histogram match for all the x's

In [ ]:
hm = pd.DataFrame(index=f_filtered['x'].values, columns=['hm1', 'hm2'])

for x in hm.index:
    hm.at[x, 'hm1'] = histogram_match(np.array(f_filtered.sel(x=x)['T']), np.array(meteo1_filtered['Lompolo']), lims=[0,30])
    hm.at[x, 'hm2'] = histogram_match(np.array(f_filtered.sel(x=x)['T']), np.array(meteo1_filtered['Kenttarova']), lims=[0,30])

In [ ]:
plt.scatter(hm.index, hm['hm1'], s=3, label='Lompolo')
plt.scatter(hm.index, hm['hm2'], s=3, label='Kenttarova')
plt.legend()
plt.ylabel('Histogram match')
plt.xlabel('Distance along stream [m]')

### Filtering those x's that have high histogram match

In [ ]:
f_data_2 = f_data.copy()
hm_lim = 0.7

filter_xs = np.array(hm.loc[(hm['hm1'] > hm_lim) | (hm['hm2'] > hm_lim)].index)
filter_xs_set = set(filter_xs)
f_data_2['T'] = xr.where(np.isin(f_data_2['x'], list(filter_xs_set)), np.nan, f_data_2['T'])

### Once again a bit better data but in some locations DTS shows much higher temperatures than meteorological observations

In [ ]:
time_slice = slice('2022-06-08', '2022-06-10')
x_slice = slice(0, 2000)
plot_2D_dts_colormap(xr_data=f_data_2, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice)

## #3 Finding DTS sections where summer temperatures are unrealistically high
#### Subsetting for summer months and common dates

In [ ]:
f_filtered_2 = f_data_2.where((f_data_2.time.dt.month > 5) & (f_data_2.time.dt.month < 9), drop=True)

# Find common times
common_times = meteo1.index.intersection(f_filtered_2.time.values)

# Subset both datasets to only include common times
f_filtered_2 = f_filtered_2.sel(time=common_times)
meteo1_filtered_2 = meteo1.loc[common_times]

#### Example distribution of way too high DTS temperature

In [ ]:
x_temp = 551.667

#plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Lompolo'], bins=50, alpha=0.5, label='Lompolo')
plt.hist(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Kenttarova'], bins=50, alpha=0.5, label='Kenttarova')
plt.hist(f_filtered_2.sel(x=x_temp)['T'], bins=50, alpha=0.5, label='DTS')
hm_temp = histogram_match(np.array(f_filtered_2.sel(x=x_temp)['T']), np.array(meteo1_filtered['Kenttarova']), lims=[0,30])
meteo_mean = round(meteo1.loc[(meteo1.index.month > 5) & (meteo1.index.month < 9), 'Kenttarova'].mean(),1)
dts_mean = round(np.nanmean(f_filtered_2.sel(x=x_temp)['T']), 1)
plt.axvline(meteo_mean, color='tab:blue', linestyle='-', label=f'mean = {meteo_mean}')
plt.axvline(dts_mean, color='tab:orange', linestyle='-', label=f'mean = {dts_mean}')
plt.title(f'X = {x_temp}, histogram match = {hm_temp}')
plt.legend()

#### Calculating and plotting summertime means for all x's and meteorological observations

In [ ]:
means = pd.DataFrame(index=f_filtered['x'].values, columns=['kenttarova_mean', 'lompolo_mean', 'dts_mean'])

for x in means.index:
    means.at[x, 'kenttarova_mean'] = np.nanmean(meteo1_filtered_2['Kenttarova'])
    means.at[x, 'lompolo_mean'] = np.nanmean(meteo1_filtered_2['Lompolo'])
    means.at[x, 'dts_mean'] = np.nanmean(f_filtered_2.sel(x=x)['T'])
means.plot()

#### Filtering those x's that have higher mean than that of air

In [ ]:
f_data_3 = f_data_2.copy()

filter_xs_2 = np.array(means.loc[((means['dts_mean'] > means['lompolo_mean']) | 
                                  (means['dts_mean'] > means['kenttarova_mean']))].index)
filter_xs_set_2 = set(filter_xs_2)
f_data_3['T'] = xr.where(np.isin(f_data_3['x'], list(filter_xs_set_2)), np.nan, f_data_3['T'])

In [ ]:
time_slice = slice('2022-06-01', '2022-08-01')
x_slice = slice(0, 2000)
plot_2D_dts_colormap(xr_data=f_data_3, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-08-01', '2022-08-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=f_data_3, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

In [ ]:
time_slice = slice('2021-12-01', '2022-03-01')
x_slice = slice(500, 1000)
plot_2D_dts_colormap(xr_data=f_data_3, meteo_df=meteo1, time_slice=time_slice, vmin=-10, vmax=10, x_slice=x_slice)

In [ ]:
time_slice = slice('2022-02-01', '2022-02-02')
x_slice = slice(50, 2000)

plot_dts_meteo_distributions(xr_data=f_data_3, meteo_df=meteo1, time_slice=time_slice, x_slice=x_slice, save_fp=None)

### Looks pretty OK now although there are still some frozen sections during low flow
#### Saving into a new .nc file

In [ ]:
out_fp = r'/Users/jpnousu/DTS_data/PROCESSED_DATA/pallas_dts_data_f_3.nc'
f_data_3.to_netcdf(out_fp)